In [1]:
import pandas as pd
import requests
import time
import glob
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pickle
import re
import math
import datetime
import pytz
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3757201255.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import pandas as pd
import requests
import time
import glob
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\4131587841.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import sys
sys.path.append('utils')
import UTILS as U
from UTILS import *

In [4]:
OUTPUT_FOLDER = "0_isw_html"
INPUT_DATA_FOLDER = 'SHIIIIIIIIIIIIIIIIT'
OUTPUT_FOLDER_CSV = 'info_for_prediction'
OUTPUT_DATA_FILE_CSV = '0_predicted_isw'
INPUT_FOLDER_ML = 'models'

In [5]:
OUTPUT_FOLDER = "SHIIIIIIIIIIIIIIIIT/0_isw_html"
BASA_URL = "https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment"

def save_page(url, file_name):
    temp_url = url
    temp_url = temp_url.split('-') 
    if temp_url[-2] < '10':
        url = url[:len(url)-5]
        if 'january' in url:
            url = url.replace('0','') + '-2023'
        else:
            url = url.replace('0','')
    print(url)
    page = requests.get(url)
    url_name = url.split("/")[-1].replace("-", "_")
    with open(f"{OUTPUT_FOLDER}/{file_name}__{url_name}.html", 'wb+') as f:
        f.write(page.content)

def get_prev_date(what):
    mnths = ["january", "february", "march", "april", "may", "june", "july", "august","september", "october", 
             "november", "december"]
    base = datetime.strptime(datetime.now().strftime("%Y-%m-%d"), "%Y-%m-%d") - timedelta(days=1)
    if what == "m":
        return mnths[int(base.strftime("%m"))-1]
    elif what == "d":
        return base.strftime("%d")
    elif what == "year":
        return base.strftime("%Y")

month = get_prev_date("m")
day = get_prev_date("d")
year = get_prev_date("year")
url = f"{BASA_URL}-{month}-{day}-{year}"
file_name = f"{month}_{day}_{year}"
import os

folder_path = f"isw_data/0_isw_html"
file_name_2 =  file_name+'__russian_offensive_campaign_assessment_'+file_name+'.html'

if not os.path.isfile(os.path.join(folder_path, file_name_2)):
    save_page(url, file_name)

https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-25-2023


In [6]:
INPUT_DATA_FOLDER = 'SHIIIIIIIIIIIIIIIIT/0_isw_html'
by_dates = glob.glob(f'{OUTPUT_FOLDER}/*.html')

In [7]:
all_data = []

for i in by_dates:
    d={}
    
    file_name = i.split('\\')[-1].split('__')
    date = datetime.strptime(file_name[0], '%B_%d_%Y')
    url = file_name[1].split('.')[0]
    
    with open(i, 'r', encoding="utf-8", errors='namereplace') as cfile:
        parsed_html = BeautifulSoup(cfile, 'html.parser')
        try:
            title = parsed_html.head.find('title').text
        except AttributeError:
            title = ""
        try:
            link = parsed_html.head.find('link', attrs={'rel':"canonical"}, href = True).attrs["href"]
        except (AttributeError, KeyError):
            link = ""
        try:
            text_title = parsed_html.body.find('h1', attrs={'id':'page-title'}).text
        except AttributeError:
            text_title = ""
        try:
            text_main = parsed_html.body.find('div', attrs={'class':'field field-name-body field-type-text-with-summary field-label-hidden'}).decode_contents(formatter="html")
        except AttributeError:
            text_main = ""
       
        dictionary = {
            'date':date,
            'short_url':url,
            'title':title,
            'text_title':text_title,
            'full_url':link,
            'main_html':text_main
        }
        
        all_data.append(dictionary)

In [8]:
df = pd.DataFrame.from_dict(all_data)

In [9]:
df.head()

,date,short_url,title,text_title,full_url,main_html
0,2023-04-25,russian_offensive_campaign_assessment_april_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field-items""><div class=""field-ite..."


In [10]:
from utils import UTILS
df['main_html_v2'] = df['main_html'].apply(lambda x: UTILS.remove_names_and_dates(x, BeautifulSoup))

C:\Users\Daryna Semenets\python_arbeiten\.venv\Scripts\Project\utils\UTILS.py:124: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 124 of the file C:\Users\Daryna Semenets\python_arbeiten\.venv\Scripts\Project\utils\UTILS.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(page_html_text)


In [11]:
pattern = '\[(\d+)\]'

df['main_html_v3'] = df['main_html_v2'].apply(lambda x: re.sub(pattern, '', x))

In [12]:
df['main_html_v4'] = df['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)

In [13]:
df['main_html_v5'] = df['main_html_v4'].apply(lambda x: re.sub(r'http(\S+.*\s)','',x))
df['main_html_v6'] = df['main_html_v5'].apply(lambda x: re.sub(r'(o2022|o2023|2022|2023)','',x))

In [14]:
df['main_html_v7'] = df['main_html_v6']


In [15]:
def del_clickHere(df):
    divider = "in this report."
    for i in range(df.shape[0]):
        if divider in df.loc[i,'main_html_v6']:
            temp_str = df.loc[i,'main_html_v6']
            temp_list = temp_str.split(divider)
            df.loc[i,'main_html_v7'] = temp_list[1]

In [16]:
del_clickHere(df)

In [17]:
def del_clickHere(df):
    divider = "invasion of Ukraine."
    for i in range(df.shape[0]):
        if divider in df.loc[i,'main_html_v7']:
            temp_str = df.loc[i,'main_html_v7']
            temp_list = temp_str.split(divider)
            df.loc[i,'main_html_v7'] = temp_list[1]

In [18]:
del_clickHere(df)

In [19]:
df['main_html'] = df['main_html'].apply(lambda x: BeautifulSoup(x).text)
df2 = df.drop(['main_html_v2','main_html_v3','main_html_v4',
               'main_html_v5', 'main_html_v6'], axis = 1)

In [20]:
def preprocess(data, word_root_algo= "lemm"):
    from UTILS import remove_one_letter_word,remove_url_string, convert_lower_case, remove_punctuation, remove_apostrophe, remove_stop_words, conver_numbers, stemming, lemmatizing
    data = UTILS.remove_one_letter_word(data)
    data = UTILS.remove_url_string(data)
    data = UTILS.convert_lower_case(data)
    data = UTILS.remove_punctuation(data)
    data = UTILS.remove_apostrophe(data)
    data = UTILS.remove_stop_words(data)
    data = UTILS.conver_numbers(data)
    data = UTILS.stemming(data)
    data = UTILS.remove_punctuation(data)
    data = UTILS.conver_numbers(data)
    
    if word_root_algo == "lemm":
        data = UTILS.lemmatizing(data)
    else:
        data = UTILS.stemming(data)
        
    data = UTILS.remove_punctuation(data)
    data = UTILS.remove_stop_words(data)
    
    return data

In [21]:
df2['report_text_lemm']= df2['main_html_v7'].apply(lambda x: preprocess(x,'lemm'))

In [22]:
df2.to_csv(f'{OUTPUT_FOLDER_CSV}/{OUTPUT_DATA_FILE_CSV}', sep =';', index = False)

In [23]:
docs = df2['report_text_lemm'].tolist()

In [24]:
cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

In [25]:
with open('models/pred_count_vectorizer_v1.pkl', 'wb') as handle:
    pickle.dump(cv ,handle)

In [26]:
tfidf_transformer =  TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [27]:
with open('models/pred_tfidf_transformer_v1.pkl', 'wb') as handle:
    pickle.dump(tfidf_transformer, handle)

In [28]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index = cv.get_feature_names_out(), columns = ['idf_weights'])


In [29]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)
tf_idf_vector.shape

(1, 828)

In [30]:
tfidf = pickle.load(open('models/pred_tfidf_transformer_v1.pkl', 'rb'))
cv = pickle.load(open('models/pred_count_vectorizer_v1.pkl', 'rb'))

In [31]:
def conver_doc_to_vector(doc):
    feature_names = cv.get_feature_names_out()
    top_n = 100
    tf_idf_vector = tfidf.transform(cv.transform([doc]))
    
    sorted_items = tf.sort_coo(tf_idf_vector.tocoo())
    
    keywords = tf.extract_topn_from_vector(feature_names, sorted_items,top_n)
    
    return keywords

In [32]:
from utils import tf
df2['keywords'] = df2['report_text_lemm'].apply(lambda x: conver_doc_to_vector(x))

In [33]:

df2['report_date']=df2['date']
df2['date_tmrw'] = df2['report_date'].apply(lambda x: x+timedelta(days=1))

In [34]:
isw_df = df2[["report_date", "date_tmrw", "keywords", "main_html_v7", "report_text_lemm"]].copy().add_prefix('isw_')

# here


In [35]:
import urllib.request
import sys
import datetime
import json
import pytz
import os
import scipy
import pandas as pd             

API_KEY = "V8J7UYNGWSQ65E3KGUDZ4YKF4"

DIR_REGIONS = 'Homework3/raw_csv/regions.csv'
SAVED_FORCASTS = 'SHIIIIIIIIIIIIIIIIT/saved_weather'


df_regions = pd.read_csv(DIR_REGIONS)

def save_file(data,city, date):
    data_object = json.dumps(data)

    # open file for writing, "w" 
    f = open(f"{SAVED_FORCASTS}/{city}_{date}.json","w")

    # write json object to file
    f.write(data_object)

    # close file
    f.close()


def read_file(path):
    f = open(path)
  
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
  
  
    # Closing file
    f.close()
    return data

def get_weather(city, date):
    
    path = f"{SAVED_FORCASTS}/{city}_{date}.json"
    if (os.path.exists(path)):
        jsonData = read_file(path)
        return jsonData
    location = f"{city},Ukraine"
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date}?key={API_KEY}&include=hours&unitGroup=metric&contentType=json'
    try: 
      ResultBytes = urllib.request.urlopen(url)
  
      # Parse the results as JSON
      jsonData = json.load(ResultBytes)

        
    except urllib.error.HTTPError  as e:
      ErrorInfo= e.read().decode() 
      print('Error code: ', e.code, ErrorInfo)
      sys.exit()
    except  urllib.error.URLError as e:
      ErrorInfo= e.read().decode() 
      print('Error code: ', e.code,ErrorInfo)
      sys.exit()
    save_file(jsonData,city, date)
    return jsonData



def get_next_date(date):
    return (date+datetime.timedelta(days=1)).strftime("%Y-%m-%d")


def get_df_weather(jsonData):
    df_data_day = pd.DataFrame(jsonData['days'])
    df_data_day = df_data_day[df_data_day.columns[0:33]].add_prefix('day_')
    hours_forecast=jsonData['days'][0]['hours']
    df_weather_hours = pd.DataFrame(hours_forecast).add_prefix('hour_')
    df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
    df_weather_hours['key'] = 1
    df_data_day['key'] = 1
    df_weather_final = pd.merge(df_data_day,df_weather_hours, on='key')
    return df_weather_final


def get_weather_for_12_hours(city,date):
    jsonData = get_weather(city, date.strftime("%Y-%m-%d"))
    current_hour = int(date.strftime("%H"))
    weather_all_data_day1 = get_df_weather(jsonData)
    hours_needed = (weather_all_data_day1['hour_int']>=current_hour)&(weather_all_data_day1['hour_int']<=(current_hour+12))
    weather_all_data_day1=weather_all_data_day1[hours_needed]
    df_weather_final = weather_all_data_day1
    hours_left=12-weather_all_data_day1.shape[0]
    if(hours_left>0):
        jsonData = get_weather(city, get_next_date(date))
        weather_all_data_day2 = get_df_weather(jsonData)
        hours_needed_2 = ((weather_all_data_day2['hour_int']<=hours_left))
        weather_all_data_day2=weather_all_data_day2[hours_needed_2]
        df_weather_final = pd.concat([weather_all_data_day1, weather_all_data_day2], axis=0)
    df_weather_final['city']=city
    df_final = pd.merge(df_weather_final,df_regions,left_on="city",right_on="center_city_en")


    return df_final




# УВАГА
тут векторайзер той самий що для тренування моделі

In [36]:
tf_idf_model = pickle.load(open ("models/tfidf_transformer_v1.pkl", "rb"))
cv_model = pickle.load(open ("models/count_vectorizer_v1.pkl", "rb"))


In [37]:
#cv_vector_model = cv_model.transform(isw_df['isw_report_text_lemm'].values.astype('U'))

In [38]:
#TF_IDF_MODEL = tf_idf_model.transform(cv_vector_model)


# MAIN PART

In [69]:
INPUT_MODELS = 'training_models'
name = 'random_forest'
version = '1.1.pkl'
name_model = f'{name}__{version}'
model_4 = pickle.load(open (f"{INPUT_MODELS}/{name_model}.pkl", "rb"))

In [68]:
cities = ['Vinnytsia','Simferopol','Lutsk','Dnipro','Donetsk','Zhytomyr','Uzhgorod','Zaporozhye','Ivano-Frankivsk','Kyiv','Kropyvnytskyi',
         'Luhansk','Lviv','Mykolaiv','Odesa','Poltava','Rivne','Sumy','Ternopil','Kharkiv','Kherson','Khmelnytskyi',
         'Cherkasy','Chernivtsi','Chernihiv']
date = datetime.datetime.now(pytz.timezone('Europe/Kyiv'))
result = {}
for city in cities:
    df_weather_final = get_weather_for_12_hours(city,date)
    df_weather_final['key']=1
    isw_df['key']=1
    df_all = df_weather_final.merge(isw_df, how = 'left', left_on = 'key', right_on = 'key')
    to_drop=['key','isw_report_date','isw_date_tmrw','isw_keywords','isw_main_html_v7','isw_report_text_lemm']
    df_weather_matrix_v1 = df_all.drop(to_drop, axis = 1)
    df_weather_matrix_v1= df_weather_matrix_v1[['day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
           'day_precip', 'day_precipcover', 'day_solarradiation',
           'day_solarenergy', 'day_uvindex', 'hour_temp', 'hour_humidity',
           'hour_dew', 'hour_precip', 'hour_precipprob', 'hour_snow',
           'hour_snowdepth', 'hour_windgust', 'hour_windspeed', 'hour_winddir',
           'hour_pressure', 'hour_visibility', 'hour_cloudcover',
           'hour_solarradiation', 'hour_uvindex', 'hour_severerisk','region_id']]
    cv_vector_model = cv_model.transform(df_all['isw_report_text_lemm'].values.astype('U'))
    TF_IDF_MODEL = tf_idf_model.transform(cv_vector_model)
    df_weather_matrix_v1_csr = scipy.sparse.csr_matrix(df_weather_matrix_v1.values)
    df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v1_csr, TF_IDF_MODEL), format='csr')
    predicted = model_4.predict(df_all_data_csr)
    current_time = datetime.datetime.now(pytz.timezone('Europe/Kyiv'))
    hours = []
    for i in range(12):
        hour = date + datetime.timedelta(hours=i)
        hour_rounded = hour.replace(minute=0, second=0, microsecond=0)
        hours.append(hour_rounded.strftime('%H:%M'))

    result[city] = dict(zip(hours, predicted))


C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_da

C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\Daryna Semenets\AppData\Local\Temp\ipykernel_6260\3537637949.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_da

In [66]:
result

{'Vinnytsia': {'20:00': False,
  '21:00': False,
  '22:00': False,
  '23:00': False,
  '00:00': False,
  '01:00': False,
  '02:00': False,
  '03:00': False,
  '04:00': False,
  '05:00': False,
  '06:00': False,
  '07:00': False},
 'Simferopol': {'20:00': False,
  '21:00': False,
  '22:00': False,
  '23:00': False,
  '00:00': False,
  '01:00': False,
  '02:00': False,
  '03:00': False,
  '04:00': False,
  '05:00': False,
  '06:00': False,
  '07:00': False},
 'Lutsk': {'20:00': False,
  '21:00': False,
  '22:00': False,
  '23:00': False,
  '00:00': False,
  '01:00': False,
  '02:00': False,
  '03:00': False,
  '04:00': False,
  '05:00': False,
  '06:00': False,
  '07:00': False},
 'Dnipro': {'20:00': False,
  '21:00': False,
  '22:00': False,
  '23:00': False,
  '00:00': False,
  '01:00': False,
  '02:00': False,
  '03:00': False,
  '04:00': False,
  '05:00': False,
  '06:00': False,
  '07:00': False},
 'Donetsk': {'20:00': False,
  '21:00': False,
  '22:00': False,
  '23:00': False,
  '

### MODEL PART